# Iniciando Sessão Spark

In [1]:
from pyspark.sql import SparkSession

# Iniciar a sessão Spark com a configuração do Delta Lake
spark = (SparkSession.builder
    .appName("DeltaLake")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Extensão do Delta
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Catálogo do Delta
    .config("spark.jars", "/usr/local/spark/jars/delta-spark_2.12-3.0.0.jar,/usr/local/spark/jars/delta-storage-3.0.0.jar")  # Caminho dos JARs no Docker
    .getOrCreate())
"""    .config("spark.executor.cores", "2")  # Limitar o número de núcleos por executor
    .config("spark.driver.cores", "1")    # Limitar o número de núcleos para o driver
    .config("spark.executor.memory", "4g")
    .config("spark.driver.memory", "4g")
    .config("spark.memory.fraction", "0.3")  # Ajuste para controlar a fração de memória usada pelo Spark
    .config("spark.memory.storageFraction", "0.2")  # Fração da memória usada para armazenar dados persistentes"""


# Verificar se a sessão Spark foi criada com sucesso
print(spark.version)

3.5.0


# Importando CSV

In [2]:
df_test = spark.read.csv("/home/jovyan/app/data/raw/temas_ambientais.csv", sep=";", header=True)
df_test.show()

# Salvando Bronze (Distribuída)

In [3]:
df_test = df_test.dropDuplicates()

In [ ]:
df_test.write.mode("overwrite").format("delta").save("/home/jovyan/app/data/bronze/bronze_distribuida")

# Limpeza e Salvando Silver (Distribuída)

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
df_test = spark.read.format("delta").load("/home/jovyan/app/data/bronze/bronze_distribuida")
df_test.show(10)

+---+-----------------+-----------+--------------+--------------------+-----------------+--------------------+------------+----------------------------------+---------------------------+---------------------------+---------------------+----------------------+-----------------+-----------------+--------------------+--------------------------------+----------------------+----------------------------+-----------------+---------------+-----------------+---------------------------+----------------------------------------+-----------+-----------------------+
| uf|        municipio|codigo_ibge|area_do_imovel|        registro_car|situacao_cadastro|   condicao_cadastro|area_liquida|area_remanescente_vegetacao_nativa|area_reserva_legal_proposta|area_preservacao_permanente|area_nao_classificada|solicitacao_adesao_pra|         latitude|        longitude|      data_inscricao|data_alteracao_condicao_cadastro|area_rural_consolidada|area_servidao_administrativa|tipo_imovel_rural|modulos_fiscais|area_u

In [5]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_1 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_1

{'uf': 0,
 'municipio': 0,
 'codigo_ibge': 0,
 'area_do_imovel': 0,
 'registro_car': 0,
 'situacao_cadastro': 0,
 'condicao_cadastro': 0,
 'area_liquida': 60416,
 'area_remanescente_vegetacao_nativa': 60416,
 'area_reserva_legal_proposta': 60416,
 'area_preservacao_permanente': 60416,
 'area_nao_classificada': 60416,
 'solicitacao_adesao_pra': 0,
 'latitude': 0,
 'longitude': 0,
 'data_inscricao': 12,
 'data_alteracao_condicao_cadastro': 5132886,
 'area_rural_consolidada': 60416,
 'area_servidao_administrativa': 60416,
 'tipo_imovel_rural': 0,
 'modulos_fiscais': 0,
 'area_uso_restrito': 60416,
 'area_reserva_legal_averbada': 60416,
 'area_reserva_legal_aprovada_nao_averbada': 60416,
 'area_pousio': 60416,
 'data_ultima_retificacao': 0}

In [6]:
df_test.printSchema()

root
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- codigo_ibge: string (nullable = true)
 |-- area_do_imovel: string (nullable = true)
 |-- registro_car: string (nullable = true)
 |-- situacao_cadastro: string (nullable = true)
 |-- condicao_cadastro: string (nullable = true)
 |-- area_liquida: string (nullable = true)
 |-- area_remanescente_vegetacao_nativa: string (nullable = true)
 |-- area_reserva_legal_proposta: string (nullable = true)
 |-- area_preservacao_permanente: string (nullable = true)
 |-- area_nao_classificada: string (nullable = true)
 |-- solicitacao_adesao_pra: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- data_inscricao: string (nullable = true)
 |-- data_alteracao_condicao_cadastro: string (nullable = true)
 |-- area_rural_consolidada: string (nullable = true)
 |-- area_servidao_administrativa: string (nullable = true)
 |-- tipo_imovel_rural: string (nullable = true)


In [7]:
df_test.columns

['uf',
 'municipio',
 'codigo_ibge',
 'area_do_imovel',
 'registro_car',
 'situacao_cadastro',
 'condicao_cadastro',
 'area_liquida',
 'area_remanescente_vegetacao_nativa',
 'area_reserva_legal_proposta',
 'area_preservacao_permanente',
 'area_nao_classificada',
 'solicitacao_adesao_pra',
 'latitude',
 'longitude',
 'data_inscricao',
 'data_alteracao_condicao_cadastro',
 'area_rural_consolidada',
 'area_servidao_administrativa',
 'tipo_imovel_rural',
 'modulos_fiscais',
 'area_uso_restrito',
 'area_reserva_legal_averbada',
 'area_reserva_legal_aprovada_nao_averbada',
 'area_pousio',
 'data_ultima_retificacao']

In [8]:
int_columns = ["codigo_ibge"]
float_columns = ["area_do_imovel", "area_liquida", "area_remanescente_vegetacao_nativa", 
                 "area_reserva_legal_proposta", "area_preservacao_permanente", "area_nao_classificada",
                 "latitude", "longitude", "area_rural_consolidada", "area_servidao_administrativa",
                 "modulos_fiscais", "area_uso_restrito", "area_reserva_legal_averbada",
                 "area_reserva_legal_aprovada_nao_averbada", "area_pousio"]
bool_columns = ["solicitacao_adesao_pra"]
date_columns = ["data_inscricao", "data_alteracao_condicao_cadastro", "data_ultima_retificacao"]
text_column = [x for x in df_test.columns if x not in int_columns+float_columns+bool_columns+date_columns]

In [9]:
# Dados Para Integer
df_test = df_test.withColumn("codigo_ibge", col("codigo_ibge").cast(IntegerType()))

# Dados Para Float
for i in float_columns:
    df_test = df_test.withColumn(i, col(i).cast(FloatType()))

# Dados para Bool
df_test = df_test.withColumn("solicitacao_adesao_pra", 
                             when(col("solicitacao_adesao_pra") == "Sim", True).
                             when(col("solicitacao_adesao_pra") == "Nao", False).
                             otherwise(None))

# Dados para Date
for i in date_columns:
    df_test = df_test.withColumn(i, to_timestamp(regexp_replace(col(i), r"\.\d+$", ""), "yyyy-MM-dd HH:mm:ss"))

df_test.printSchema()

root
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- codigo_ibge: integer (nullable = true)
 |-- area_do_imovel: float (nullable = true)
 |-- registro_car: string (nullable = true)
 |-- situacao_cadastro: string (nullable = true)
 |-- condicao_cadastro: string (nullable = true)
 |-- area_liquida: float (nullable = true)
 |-- area_remanescente_vegetacao_nativa: float (nullable = true)
 |-- area_reserva_legal_proposta: float (nullable = true)
 |-- area_preservacao_permanente: float (nullable = true)
 |-- area_nao_classificada: float (nullable = true)
 |-- solicitacao_adesao_pra: boolean (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- data_inscricao: timestamp (nullable = true)
 |-- data_alteracao_condicao_cadastro: timestamp (nullable = true)
 |-- area_rural_consolidada: float (nullable = true)
 |-- area_servidao_administrativa: float (nullable = true)
 |-- tipo_imovel_rural: string (nullable = true)
 |

In [10]:
# Criar uma lista com o nome das colunas e o número de nulos em cada uma
null_counts_2 = df_test.select([sum(col(c).isNull().cast("int")).alias(c) for c in df_test.columns]).first().asDict()

# Exibir o resultado
null_counts_2

{'uf': 0,
 'municipio': 0,
 'codigo_ibge': 0,
 'area_do_imovel': 0,
 'registro_car': 0,
 'situacao_cadastro': 0,
 'condicao_cadastro': 0,
 'area_liquida': 60416,
 'area_remanescente_vegetacao_nativa': 60416,
 'area_reserva_legal_proposta': 60416,
 'area_preservacao_permanente': 60416,
 'area_nao_classificada': 60416,
 'solicitacao_adesao_pra': 0,
 'latitude': 0,
 'longitude': 0,
 'data_inscricao': 12,
 'data_alteracao_condicao_cadastro': 5132886,
 'area_rural_consolidada': 60416,
 'area_servidao_administrativa': 60416,
 'tipo_imovel_rural': 0,
 'modulos_fiscais': 0,
 'area_uso_restrito': 60416,
 'area_reserva_legal_averbada': 60416,
 'area_reserva_legal_aprovada_nao_averbada': 60416,
 'area_pousio': 60416,
 'data_ultima_retificacao': 0}

In [11]:
if null_counts_1 == null_counts_2:
    print("Os valores de nulos são os mesmos em ambas as versões.")
else:
    print("Há diferenças nos valores de nulos.")
    # Mostrar as diferenças
    for coluna in null_counts_1.keys():
        if null_counts_1[coluna] != null_counts_2[coluna]:
            print(f"Coluna '{coluna}': null_counts_1 = {null_counts_1[coluna]}, null_counts_2 = {null_counts_2[coluna]}")

Os valores de nulos são os mesmos em ambas as versões.


# Salvando Silver (Distribuída)

In [12]:
df_test.write.mode("overwrite").format("delta").save("/home/jovyan/app/data/silver/silver_distribuida")

# Salvando Gold (Distribuída)